In [1]:
!pip install -q nltk transformers sentencepiece torch tensorflow

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import pandas as pd
import numpy as np
import nltk
from nltk.translate.bleu_score import sentence_bleu
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from sklearn.model_selection import train_test_split
from transformers import MarianMTModel, MarianTokenizer

2026-01-24 16:40:22.272183: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769272822.712691      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769272822.830745      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769272824.010987      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769272824.011034      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769272824.011037      55 computation_placer.cc:177] computation placer alr

In [2]:
import pandas as pd
data = pd.read_csv('/kaggle/input/en-bn-csv/en_bn.csv')
data.columns = ['en_text', 'bn_text']
data.head()

,en_text,bn_text
0,"Hello, good to see you!","হ্যালো, আপনাকে দেখে ভালো লাগলো!"
1,"Goodbye, have a great day!","বিদায়, আপনার দিন শুভ হোক!"
2,Thank you very much for your help.,আপনার সহায়তার জন্য অনেক ধন্যবাদ।
3,I appreciate it.,আমি এটি প্রশংসা করি।
4,You're welcome!,আপনার স্বাগতম!


## 4. Preprocessing & Tokenization

In [3]:
eng_tokenizer = Tokenizer()
bn_tokenizer = Tokenizer()

eng_tokenizer.fit_on_texts(data['en_text'])
bn_tokenizer.fit_on_texts(data['bn_text'])

X = pad_sequences(eng_tokenizer.texts_to_sequences(data['en_text']), padding='post')
y = pad_sequences(bn_tokenizer.texts_to_sequences(data['bn_text']), padding='post')

## 5. Train-Test Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## 6. Seq2Seq LSTM Model

In [5]:
latent_dim = 256

encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(len(eng_tokenizer.word_index)+1, latent_dim)(encoder_inputs)
_, h, c = LSTM(latent_dim, return_state=True)(enc_emb)
encoder_states = [h, c]

decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(len(bn_tokenizer.word_index)+1, latent_dim)(decoder_inputs)
dec_outputs, _, _ = LSTM(latent_dim, return_sequences=True, return_state=True)(dec_emb, initial_state=encoder_states)
dec_outputs = Dense(len(bn_tokenizer.word_index)+1, activation='softmax')(dec_outputs)

seq2seq_model = Model([encoder_inputs, decoder_inputs], dec_outputs)
seq2seq_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
seq2seq_model.summary()

I0000 00:00:1769272864.374104      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1769272864.378028      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 256) │     31,744 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 256) │     32,512 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    525,312 │ embedding[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │    525,312 │ embedding_1[0][0… │
│                     │ 256), (None,      │            │ lstm[0][1],       │
│                     │ 256), (None,      │            │ lstm[0][2]        │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 127) │     32,639 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,147,519 (4.38 MB)

 Trainable params: 1,147,519 (4.38 MB)

 Non-trainable params: 0 (0.00 B)

## 7. Train Seq2Seq Model (Demo Epochs)

In [6]:
seq2seq_model.fit(
    [X_train, y_train[:, :-1]],
    y_train[:, 1:],
    epochs=3,
    batch_size=64
)

Epoch 1/3


I0000 00:00:1769272869.998083     128 cuda_dnn.cc:529] Loaded cuDNN version 91002


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.5279
Epoch 2/3
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 2.4723e-04
Epoch 3/3
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 6.4850e-05


In [7]:
!pip install -U transformers sentencepiece sacremoses accelerate
!pip install -U huggingface_hub
!huggingface-cli login
!pip uninstall -y transformers huggingface-hub tokenizers
!pip install transformers huggingface-hub sentencepiece
!pip install -U huggingface_hub
!huggingface-cli login
!pip install -U transformers huggingface-hub sentencepiece
!pip install "pydantic<2.12"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 117.7 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 10.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
ERROR: pip's dependency resolver does n

****Transformer Model (MarianMT)¶
BLEU Score Evaluation¶****

In [8]:
!pip install -q transformers sentencepiece sacrebleu
!pip install sacrebleu
import torch
from transformers import MT5Tokenizer, MT5ForConditionalGeneration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 3.5 MB/s eta 0:00:00


Neural Machine Translation (NMT) model

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

translation_model_name = "google/mt5-small"  

translation_tokenizer = AutoTokenizer.from_pretrained(translation_model_name)
translation_model = AutoModelForSeq2SeqLM.from_pretrained(translation_model_name)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
import pandas as pd
import sacrebleu
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from tqdm import tqdm
import re

model_name = "facebook/nllb-200-1.3B"

print(f"Loading {model_name}... (This might take a minute)")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def clean_text(text):
    if not isinstance(text, str): return ""
    text = text.replace('\n', ' ')  # Remove newlines
    text = re.sub(' +', ' ', text)  # Remove double spaces
    return text.strip()

def nllb_translate_advanced(text):
    if not text: return ""
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
    
    bos_token_id = tokenizer.convert_tokens_to_ids("ben_Beng")
    
    translated_tokens = model.generate(
        **inputs, 
        forced_bos_token_id=bos_token_id, 
        max_length=128,
        num_beams=10,            # High beam count for quality
        no_repeat_ngram_size=3,  # Prevents repeating phrases
        length_penalty=1.0       # Balances length
    )
    
    return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

try:
    df = pd.read_csv('/kaggle/input/en-bn-csv/en_bn.csv')
except FileNotFoundError:
    df = pd.read_csv("en_bn.csv")

print(f"Dataset loaded: {len(df)} rows.")

print("Cleaning text...")
df['en_text'] = df['en_text'].apply(clean_text)
df['bn_text'] = df['bn_text'].apply(clean_text)

df = df[df['en_text'].str.len() > 5]
df = df[df['bn_text'].str.len() > 5]

test_data = df.sample(50, random_state=42)

sentences = test_data['en_text'].tolist()
references = test_data['bn_text'].tolist()

print(f"Translating {len(sentences)} sentences from dataset...")
hyp_nllb = []

for text in tqdm(sentences):
    hyp_nllb.append(nllb_translate_advanced(text))

print("\n--- Sample Outputs ---")
for i in range(min(5, len(sentences))):
    print(f"Input:     {sentences[i]}")
    print(f"Reference: {references[i]}")
    print(f"Predicted: {hyp_nllb[i]}")
    print("-" * 30)

bleu_score = sacrebleu.corpus_bleu(hyp_nllb, [references])
print(f"\nFinal Dataset BLEU Score: {bleu_score.score}")

Loading facebook/nllb-200-1.3B... (This might take a minute)


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Dataset loaded: 120000 rows.
Cleaning text...
Translating 50 sentences from dataset...



100%|██████████| 50/50 [00:29<00:00,  1.67it/s]


--- Sample Outputs ---
Input:     Hey, what's up?
Reference: আরে, কেমন আছ?
Predicted: হেই, কি হয়েছে?
------------------------------
Input:     Bengaluru is known as the Silicon Valley of India.
Reference: বেঙ্গালুরুকে ভারতের সিলিকন ভ্যালি বলা হয়।
Predicted: বেঙ্গালুরুকে ভারতের সিলিকন ভ্যালি বলা হয়।
------------------------------
Input:     Where is the nearest hospital?
Reference: নিকটতম হাসপাতাল কোথায়?
Predicted: নিকটতম হাসপাতাল কোথায়?
------------------------------
Input:     Please accept my heartfelt congratulations.
Reference: অনুগ্রহ করে আমার আন্তরিক অভিনন্দন গ্রহণ করুন।
Predicted: দয়া করে আমার আন্তরিক অভিনন্দন গ্রহণ করুন।
------------------------------
Input:     What is your favorite color?
Reference: আপনার প্রিয় রঙ কী?
Predicted: তোমার প্রিয় রঙ কোনটি?
------------------------------

Final Dataset BLEU Score: 31.692958920284845


In [11]:
!pip install -q -U bitsandbytes accelerate transformers
!pip install -U bitsandbytes accelerate transformers
!pip install -U transformers accelerate bitsandbytes sacrebleu pandas
!pip install -U -q bitsandbytes accelerate transformers
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U transformers
!pip install -U bitsandbytes accelerate
!pip install -U sacrebleu bitsandbytes accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 94.2 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
ydata-profiling 4.18.1 requires pandas!=1.4.0,<3.0,>1.5, but you have pandas 3.0.0 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.47.0 which is incompatible.
google-colab 1.0.0 requires jupyter-server==2.14.0, but you have jupyter-server 2.12.5 which is incompatible.

In [12]:
!pip install sacrebleu
import pandas as pd
import sacrebleu
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import re

model_name = "Qwen/Qwen2.5-3B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on: {device}")
print(f"Loading {model_name} in standard Float16 (No Quantization)...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # This replaces quantization
    device_map="auto",
    trust_remote_code=True
)

def llm_translate(text):
    if not text: return ""
    
    messages = [
        {"role": "system", "content": "You are a professional English to Bengali translator. Translate the user's text into Bengali directly. Do not provide explanations or notes."},
        {"role": "user", "content": text}
    ]
    
    text_input = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text_input], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=128,
            do_sample=False,  # Greedy decoding for accuracy
            pad_token_id=tokenizer.pad_token_id
        )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response.strip()

def clean_text(text):
    if not isinstance(text, str): return ""
    text = text.replace('\n', ' ')
    text = re.sub(' +', ' ', text)
    return text.strip()

try:
    df = pd.read_csv('/kaggle/input/en-bn-csv/en_bn.csv')
except FileNotFoundError:
    try:
        df = pd.read_csv("en_bn.csv")
    except:
        print("CSV not found. Creating dummy data.")
        df = pd.DataFrame({
            'en_text': ["I love artificial intelligence.", "The weather is beautiful today.", "Machine learning requires data."], 
            'bn_text': ["আমি কৃত্রিম বুদ্ধিমত্তা ভালোবাসি।", "আজকের আবহাওয়া চমৎকার।", "মেশিন লার্নিংয়ের জন্য ডেটা প্রয়োজন।"]
        })

print("Cleaning data...")
df['en_text'] = df['en_text'].apply(clean_text)
df['bn_text'] = df['bn_text'].apply(clean_text)

df = df[(df['en_text'].str.len() > 5) & (df['bn_text'].str.len() > 5)]

test_data = df.sample(min(50, len(df)), random_state=42)
sentences = test_data['en_text'].tolist()
references = test_data['bn_text'].tolist()

print(f"Translating {len(sentences)} sentences...")
hyp_llm = []

for text in tqdm(sentences):
    hyp_llm.append(llm_translate(text))

print("\n--- LLM vs Reference (First 3) ---")
for i in range(min(3, len(sentences))):
    print(f"Input:     {sentences[i]}")
    print(f"Reference: {references[i]}")
    print(f"LLM Pred:  {hyp_llm[i]}")
    print("-" * 30)

bleu_score = sacrebleu.corpus_bleu(hyp_llm, [references])
print(f"\nFinal LLM BLEU Score: {bleu_score.score:.2f}")

Running on: cuda
Loading Qwen/Qwen2.5-3B-Instruct in standard Float16 (No Quantization)...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Cleaning data...
Translating 50 sentences...


100%|██████████| 50/50 [02:08<00:00,  2.57s/it]


--- LLM vs Reference (First 3) ---
Input:     Hey, what's up?
Reference: আরে, কেমন আছ?
LLM Pred:  হ্যালো, আপনি কীভাবে আরও খুশিমুক্ত?
------------------------------
Input:     Bengaluru is known as the Silicon Valley of India.
Reference: বেঙ্গালুরুকে ভারতের সিলিকন ভ্যালি বলা হয়।
LLM Pred:  বঙ্গালুরু ইন্দোনেশিয়ার সিনেল ব্যালুয়ার হিসেবে পরিচিত।
------------------------------
Input:     Where is the nearest hospital?
Reference: নিকটতম হাসপাতাল কোথায়?
LLM Pred:  নিকটবর্তী সবচেয়ে নাম্বার হল কোথায়?
------------------------------

Final LLM BLEU Score: 3.04
